In [19]:
import json
import folium
import pandas
import requests
from bs4 import BeautifulSoup
import urllib.request
import geotag
import kml2geojson 
import zipfile
from xml.dom import minidom
import os

In [20]:
topic = "171384"
place_file = open("initial-10/places_"+topic+".json")
places = json.load(place_file)
path = ("./topics/" + topic)
os.mkdir(path)
os.mkdir(path + "/kmz-files")
os.mkdir(path + "/kml-files")
os.mkdir(path + "/geojsons")

# soup.
# https://gadm.org/maps.html

r = requests.get("https://gadm.org/maps.html")
htmlcontent = BeautifulSoup(r.content, "html.parser")
countries = []
country_containers = htmlcontent.find('div', class_="container-fluid main-container").p.small
# print(country_containers)

country_codes = {}
count = 'A'

for a_tag in country_containers.find_all('a', href=True):
    name = a_tag.string
    if name[0].islower():
        name = count + name
        count = chr(ord(count) + 1)
        if (count == 'X'):
            count = chr(ord(count) + 1)
    country_codes.update({name: (a_tag['href'][5:8])})


In [21]:
def get_geojson(code):
    url="https://biogeo.ucdavis.edu/data/gadm3.6/kmz/gadm36_" + code + "_0.kmz"
    
    urllib.request.urlretrieve(url, path + "/kmz-files/" + code + ".kml.zip")
    print(code)
    zf = zipfile.ZipFile(path + "/kmz-files/" + code + ".kml.zip")
    
    zf.extractall(path + "/kml-files/")
    zf.close()
    kml2geojson.main.convert(path + '/kml-files/gadm36_' + code + "_0.kml", path + "/geojsons/", separate_folders=False, style_type=None, style_filename='style.json')
    

In [22]:
# https://biogeo.ucdavis.edu/data/gadm3.6/kmz/gadm36_AFG_0.kmz
GT = geotag.Geotag()
place_names = []

for place in places:
    tag = place["geotag"]
    if (tag in GT.path2entry):
        tag = GT.path2entry[tag]
        place_names.append(tag['name'])

print(place_names)

for place in place_names:
    if (place in country_codes.keys()):
        country = country_codes.get(place)
        get_geojson(country)
            

['Vienna', 'Austria', 'Germany', 'Neuss', 'Berlin', 'Graz', 'Duesseldorf', 'France', 'Syria', 'Albania', 'Belgium', 'North Rhine-Westphalia', 'Tunisia', 'Paris', 'Salzburg']
AUT
DEU
FRA
SYR
ALB
BEL
TUN
